# Double Pendulum
Example of Lagrange and Hamilton equations of a double simple-pendulum.

## Setup
Define the constants and position vectors

In [1]:
using SymPy

In [2]:
m1, m2, l1, l2, g = symbols("m1, m2, l1, l2, g", positive=true)
t = symbols("t")
θ1 = SymFunction("theta_1")(t)
θ2 = SymFunction("theta_2")(t)
ihat = [1,0,0]
jhat = [0,1,0]
khat = [0,0,1]
;

In [3]:
r1 = l1*(sin(θ1)*ihat - cos(θ1)*jhat)

3-element Array{Sym,1}:
  l1*sin(theta_1(t))
 -l1*cos(theta_1(t))
                   0

In [4]:
r2 = r1 + l2*(sin(θ2)*ihat - cos(θ2)*jhat)

3-element Array{Sym,1}:
  l1*sin(theta_1(t)) + l2*sin(theta_2(t))
 -l1*cos(theta_1(t)) - l2*cos(theta_2(t))
                                        0

## Potential Energy
Let's make the Potential Energy $V$

In [5]:
V = m1*g*r1[2] + m2*g*r2[2]

-g*l1*m1*cos(theta_1(t)) + g*m2*(-l1*cos(theta_1(t)) - l2*cos(theta_2(t)))

## Kinetic Energy

In [6]:
T1 = 1//2*m1*sum( diff.(r1,t) .* diff.(r1,t) ) |> simplify

                       2
  2    /d             \ 
l1 *m1*|--(theta_1(t))| 
       \dt            / 
------------------------
           2            

In [7]:
T2 = 1//2*m2*sum( diff.(r2,t) .* diff.(r2,t) ) |> simplify

   /                    2                                                     
   |  2 /d             \                                         d            
m2*|l1 *|--(theta_1(t))|  + 2*l1*l2*cos(theta_1(t) - theta_2(t))*--(theta_1(t)
   \    \dt            /                                         dt           
------------------------------------------------------------------------------
                                                           2                  

                                       2\
  d                  2 /d             \ |
)*--(theta_2(t)) + l2 *|--(theta_2(t))| |
  dt                   \dt            / /
-----------------------------------------
                                         

In [8]:
T = T1 + T2

                       2      /                    2                          
  2    /d             \       |  2 /d             \                           
l1 *m1*|--(theta_1(t))|    m2*|l1 *|--(theta_1(t))|  + 2*l1*l2*cos(theta_1(t) 
       \dt            /       \    \dt            /                           
------------------------ + ---------------------------------------------------
           2                                                                  

                                                                  2\
              d              d                  2 /d             \ |
- theta_2(t))*--(theta_1(t))*--(theta_2(t)) + l2 *|--(theta_2(t))| |
              dt             dt                   \dt            / /
--------------------------------------------------------------------
        2                                                           

## Lagrange's equation

In [9]:
L = T-V

                                                                              
                                                                              
                                                                            l1
                                                                              
g*l1*m1*cos(theta_1(t)) - g*m2*(-l1*cos(theta_1(t)) - l2*cos(theta_2(t))) + --
                                                                              

                     2      /                    2                            
2    /d             \       |  2 /d             \                             
 *m1*|--(theta_1(t))|    m2*|l1 *|--(theta_1(t))|  + 2*l1*l2*cos(theta_1(t) - 
     \dt            /       \    \dt            /                             
---------------------- + -----------------------------------------------------
         2                                                                    

                                                  

In [10]:
Lag(q) = diff(L,diff(q,t),t) - diff(L,q)

Lag (generic function with 1 method)

In [11]:
Lag(θ1) |> simplify

   /                                                      2                   
   |                                                     d                    
l1*|g*m1*sin(theta_1(t)) + g*m2*sin(theta_1(t)) + l1*m1*---(theta_1(t)) + l1*m
   |                                                      2                   
   \                                                    dt                    

    2                                                                  2      
   d                                                   /d             \       
2*---(theta_1(t)) + l2*m2*sin(theta_1(t) - theta_2(t))*|--(theta_2(t))|  + l2*
    2                                                  \dt            /       
  dt                                                                          

                                  2            \
                                 d             |
m2*cos(theta_1(t) - theta_2(t))*---(theta_2(t))|
                                  2            |
            

In [12]:
Lag(θ2) |> simplify

      /                                                                    2  
      |                                                    /d             \   
l2*m2*|g*sin(theta_2(t)) - l1*sin(theta_1(t) - theta_2(t))*|--(theta_1(t))|  +
      |                                                    \dt            /   
      \                                                                       

                                   2                    2            \
                                  d                    d             |
 l1*cos(theta_1(t) - theta_2(t))*---(theta_1(t)) + l2*---(theta_2(t))|
                                   2                    2            |
                                 dt                   dt             /

## Hamilton's Equations

In [13]:
p1, p2 = symbols("p1,p2")

(p1, p2)

In [14]:
eqn = [Eq(diff(L, diff(θ1,t)), p1), Eq(diff(L, diff(θ2,t)),p2) ]

2-element Array{Sym,1}:
 Eq(l1^2*m1*Derivative(theta_1(t), t) + m2*(2*l1^2*Derivative(theta_1(t), t) + 2*l1*l2*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t))/2, p1)
                                     Eq(m2*(2*l1*l2*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) + 2*l2^2*Derivative(theta_2(t), t))/2, p2)

In [15]:
sol_p = solve(eqn, diff.([θ1,θ2],t) )

Dict{Any,Any} with 2 entries:
  Derivative(theta_2(t), t) => (-l1*p2*(m1 + m2) + l2*m2*p1*cos(theta_1(t) - th…
  Derivative(theta_1(t), t) => (l1*p2*cos(theta_1(t) - theta_2(t)) - l2*p1)/(l1…

In [16]:
H = T+V |> subs(sol_p) |> simplify

                                                             2                
        2   2    /           2                              \                 
- 2*g*l1 *l2 *m2*\m1 - m2*cos (theta_1(t) - theta_2(t)) + m2/ *(l1*m1*cos(thet
------------------------------------------------------------------------------
                                                                              
                                                                              
                                                                              

                                                                              
                                                                              
a_1(t)) + l1*m2*cos(theta_1(t)) + l2*m2*cos(theta_2(t))) + m1*m2*(l1*p2*cos(th
------------------------------------------------------------------------------
                                                                              
                                                   

In [17]:
F = simplify.( [ diff(H, p1), diff(H, p2),-diff(H, θ1), -diff(H,θ2) ] )

4-element Array{Sym,1}:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     (-l1*p2*cos(theta_1(t) - theta_2(t)) + l2*p1)/(l1^2*l2*(m1 - m2*cos(theta_1(t) - theta_2(t))^2 + m2))
     

Let's put the system of equation into state-space, and then output to Matlab compatible code.

In [18]:
z1,z2,z3,z4 = symbols("z1,z2,z3,z4")
rule1 = Dict( θ1=>z1, θ2=>z2, p1=>z3, p2=>z4 )

Dict{Sym,Sym} with 4 entries:
  p2         => z4
  p1         => z3
  theta_1(t) => z1
  theta_2(t) => z2

In [19]:
sympy.octave_code( [subs(f, rule1 ) for f in F] )

"[(-l1.*z4.*cos(z1 - z2) + l2.*z3)./(l1.^2.*l2.*(m1 - m2.*cos(z1 - z2).^2 + m2)); (l1.*m1.*z4 + l1.*m2.*z4 - l2.*m2.*z3.*cos(z1 - z2))./(l1.*l2.^2.*m2.*(m1 - m2.*cos(z1 - z2).^2 + m2)); (-g.*l1.^3.*l2.^2.*m1.^3.*sin(z1) + 2*g.*l1.^3.*l2.^2.*m1.^2.*m2.*sin(z1).*cos(z1 - z2).^2 - 3*g.*l1.^3.*l2.^2.*m1.^2.*m2.*sin(z1) - g.*l1.^3.*l2.^2.*m1.*m2.^2.*sin(z1).*cos(z1 - z2).^4 + 4*g.*l1.^3.*l2.^2.*m1.*m2.^2.*sin(z1).*cos(z1 - z2).^2 - 3*g.*l1.^3.*l2.^2.*m1.*m2.^2.*sin(z1) - g.*l1.^3.*l2.^2.*m2.^3.*sin(z1).*cos(z1 - z2).^4 + 2*g.*l1.^3.*l2.^2.*m2.^3.*sin(z1).*cos(z1 - z2).^2 - g.*l1.^3.*l2.^2.*m2.^3.*sin(z1) + l1.^2.*m1.*z4.^2.*sin(2*z1 - 2*z2)/2 + l1.^2.*m2.*z4.^2.*sin(2*z1 - 2*z2)/2 - l1.*l2.*m1.*z3.*z4.*sin(z1 - z2) - l1.*l2.*m2.*z3.*z4.*sin(z1 - z2).*cos(z1 - z2).^2 - l1.*l2.*m2.*z3.*z4.*sin(z1 - z2) + l2.^2.*m2.*z3.^2.*sin(2*z1 - 2*z2)/2)./(l1.^2.*l2.^2.*(m1.^2 - 2*m1.*m2.*cos(z1 - z2).^2 + 2*m1.*m2 + m2.^2.*cos(z1 - z2).^4 - 2*m2.^2.*cos(z1 - z2).^2 + m2.^2)); (-g.*l1.^2.*l2.^3.*m1.^2.*m2